Fine-tuning best T5 Transformer 🤖
-----------------------------------

In this notebook, we will continue the fine-tuning of T5 transformer on the new extracted sentences from the book **Grammaire de Wolof Moderne** without considering the definitions. We provide, bellow, the main evaluation figures, obtained from the hyperparameter search step. We will evaluate the training on the validation dataset.

- Parallel coordinates from panel:

- Parameter importance char: 
[t5_v3_importance](https://wandb.ai/oumar-kane-team/small-t5-cross-fw-translation-bayes-hpsearch-v3/reports/undefined-23-05-16-10-36-17---Vmlldzo0Mzc4NDY0?accessToken=eyaiyrid0qz1zg2jkq3fc65biw53084dpfitbi0dgonq6mweupw6kgjml9d2nv1w)

We can see in the above chart that the batch is the most important parameter with a negative correlation with the BLEU score (meaning that a lower batch size is better). Next, we the probability of modifying a character in the french corpus is also important and a high probability provide a better BLEU score.  

In [2]:
# let us import all necessary libraries
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5TokenizerFast, set_seed, AdamW, get_linear_schedule_with_warmup, T5ForConditionalGeneration,\
    get_cosine_schedule_with_warmup, Adafactor
from wolof_translate.utils.sent_transformers import TransformerSequences
from wolof_translate.utils.improvements.end_marks import add_end_mark # added
from torch.nn import TransformerEncoderLayer, TransformerDecoderLayer
from torch.utils.data import Dataset, DataLoader, random_split
from wolof_translate.data.dataset_v4 import SentenceDataset # v2 -> v3 -> v4
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import _LRScheduler
# from custom_rnn.utils.kwargs import Kwargs
from torch.nn.utils.rnn import pad_sequence
from plotly.subplots import make_subplots
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from torch.nn import functional as F
import plotly.graph_objects as go
from tokenizers import Tokenizer
import matplotlib.pyplot as plt
import pytorch_lightning as lt
from tqdm import tqdm, trange
from functools import partial
from torch.nn import utils
from copy import deepcopy
from torch import optim
from typing import *
from torch import nn
import pandas as pd
import numpy as np
import itertools
import evaluate
import random
import string
import shutil
import wandb
import torch
import json
import copy
import os

# add seed for everything
lt.seed_everything(0)

os.environ["WANDB_DISABLED"] = "true"

Global seed set to 0


## French to wolof

### Configure dataset 🔠

In [3]:
# recuperate the tokenizer from a json file
tokenizer = T5TokenizerFast(tokenizer_file=f"wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v3_2.json")


In [4]:
def recuperate_datasets(fr_char_p: float, fr_word_p: float, max_len: int, end_mark_opt: int):

  # Let us recuperate the end_mark adding option
  if end_mark_opt == 1:
    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space)

    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space)
    
  else:
    
    if end_mark_opt == 2:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!', replace = True)
    
    elif end_mark_opt == 3:

      end_mark_fn = partial(add_end_mark)
    
    elif end_mark_opt == 4:

      end_mark_fn = partial(add_end_mark, end_mark_to_remove = '!')

    # Create augmentation to add on French sentences
    fr_augmentation_1 = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p,
                                                             aug_word_max = max_len),
                                          remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)
    
    fr_augmentation_2 = TransformerSequences(remove_mark_space, delete_guillemet_space, add_mark_space, end_mark_fn)
    
  # Recuperate the train dataset
  train_dataset_aug = SentenceDataset(f"data/extractions/new_data/train_set.csv",
                                        tokenizer,
                                        truncation = True, max_len=max_len,
                                        cp1_transformer = fr_augmentation_1,
                                        cp2_transformer = fr_augmentation_2
                                        )

  # Recuperate the valid dataset
  valid_dataset = SentenceDataset(f"data/extractions/new_data/valid_set.csv",
                                        tokenizer, max_len=max_len,
                                        cp1_transformer = fr_augmentation_2,
                                        cp2_transformer = fr_augmentation_2,
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, valid_dataset

### Configure the model and the evaluation function ⚙️

Let us evaluate the predictions with the `bleu` metric.

Let us initialize the evaluation object.

In [5]:
%run wolof-translate/wolof_translate/utils/evaluation.py
evaluation = TranslationEvaluation(tokenizer)


### Searching for the best parameters 🕖

In [6]:
from wolof_translate.models.transformers.optimization import TransformerScheduler
from wolof_translate.trainers.transformer_trainer import ModelRunner
from wolof_translate.utils.evaluation import TranslationEvaluation
from wolof_translate.models.transformers.main import Transformer
from wolof_translate.utils.split_with_valid import split_data


-------------

### --- Wandb v5 3000

In [8]:
842//2

421

In [7]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.6723569050184816,
    'fr_word_p': 0.08544139434972758,
    'learning_rate': 0.00263707394928414,
    'weight_decay': 0.01,
    'batch_size': 16,
    'warmup_ratio': 0.0,
    'max_epoch': 842,
    'epochs': 200,
    'mid_epoch': 421,
    'max_len': 56,
    'end_mark': 4,
    'bleu': 3.8295,
    'model_dir': 'data/checkpoints/fw_t5_small_custom_train_v5_checkpoints/',
    'new_model_dir': 'data/checkpoints/t5_small_custom_train_results_fw_v5/'
}

# Initialize the model name
model_name = 't5-small'

# import the model with its pre-trained weights
model = T5ForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = 5, evaluation = evaluation, optimizer=Adafactor)

# split the data
split_data(config['random_state'], csv_file = "ad_sentences.csv")

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], config['max_len'],
                                                    config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# # Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                lr_scheduler=get_linear_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/t5_small_custom_train_fw"
                )

# We will from checkpoints so let us the model
trainer.load(config['model_dir'], load_best=True) # Only for the first loading
# trainer.load(config['new_model_dir'], load_best = True)

        

OSError: Le fichier fw_t5_small_custom_train_v5_checkpoints/best_checkpoints.pth est introuvable. Vérifiez si le chemin fourni est correct!

### ---

In [10]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/195 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 6: 
{Learning rate: [0.0026213092673829214]}


Test batch number 19: 100%|██████████| 19/19 [00:21<00:00,  1.11s/batches]



Metrics: {'train_loss': 0.5447575534383456, 'test_loss': 0.7493734171516017, 'bleu': 3.0033, 'gen_len': 11.1367}




  1%|          | 1/195 [01:08<3:41:55, 68.64s/it]

...

Test batch number 19: 100%|██████████| 19/19 [00:25<00:00,  1.32s/batches]



Metrics: {'train_loss': 0.0071780021116137505, 'test_loss': 1.196321754079116, 'bleu': 11.1321, 'gen_len': 10.5933}




 49%|████▉     | 96/195 [1:45:17<1:53:17, 68.66s/it]

For epoch 102: 
{Learning rate: [0.002318627374879521]}


Train batch number 27:  17%|█▋        | 26/150 [00:08<00:36,  3.43batches/s]

### --- Wandb v5_2 2000

127

In [8]:
# let us initialize the hyperparameter configuration 
config = {
    'random_state': 0,
    'fr_char_p': 0.06673707344553906,
    'fr_word_p': 0.07626063714036732,
    'learning_rate': 0.0030232985434635257,
    'weight_decay': 0.02659316315720403,
    'batch_size': 16,
    'warmup_ratio': 0.0,
    'max_epoch': 827,
    'epochs': 83,
    'mid_epoch': 83,
    'max_len': 19,
    'end_mark': 3,
    'bleu': 8.4915,
    'model_dir': 'fw_t5_small_custom_train_v5_2_checkpoints',
    'new_model_dir': 't5_small_custom_train_results_fw_v5_2'
}

# Initialize the model name
model_name = 't5-small'

# import the model with its pre-trained weights
model = T5ForConditionalGeneration.from_pretrained(model_name)

# resize the token embeddings
model.resize_token_embeddings(len(tokenizer))

# let us initialize the evaluation class
evaluation = TranslationEvaluation(tokenizer)

# let us initialize the trainer
trainer = ModelRunner(model, seed = 0, version = '5_2', evaluation = evaluation, optimizer=Adafactor)

# split the data
split_data(config['random_state'], csv_file = "ad_sentences.csv")

# recuperate train and test set
train_dataset, test_dataset = recuperate_datasets(config['fr_char_p'], 
                                                    config['fr_word_p'], config['max_len'],
                                                    config['end_mark'])

# let us calculate the appropriate warmup steps (let us take a max epoch of 100)
length = len(train_dataset)

n_steps = length // config['batch_size']

num_steps = config['max_epoch'] * n_steps

warmup_steps = (config['max_epoch'] * n_steps) * config['warmup_ratio']

# # Initialize the scheduler parameters
scheduler_args = {'num_warmup_steps': warmup_steps, 'num_training_steps': num_steps}

# Initialize the optimizer parameters
optimizer_args = {
    'lr': config['learning_rate'],
    'weight_decay': config['weight_decay'],
    # 'betas': (0.9, 0.98),
    'warmup_init': False,
    'relative_step': False
}

# Initialize the loaders parameters
train_loader_args = {'batch_size': config['batch_size']}

# Add the datasets and hyperparameters to trainer
trainer.compile(train_dataset, test_dataset, tokenizer, train_loader_args,
                optimizer_kwargs = optimizer_args,
                lr_scheduler=get_linear_schedule_with_warmup,
                lr_scheduler_kwargs=scheduler_args, 
                predict_with_generate = True,
                hugging_face = True,
                logging_dir="data/logs/t5_small_custom_train_fw"
                )

# We will from checkpoints so let us the model
trainer.load(config['model_dir'], load_best=True) # Only for the first loading
# trainer.load(config['new_model_dir'], load_best = True)

        

OSError: Le fichier fw_t5_small_custom_train_v5_2_checkpoints/best_checkpoints.pth est introuvable. Vérifiez si le chemin fourni est correct!

### ---

In [8]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/122 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 6: 
{Learning rate: [0.0034131613992851753]}


Test batch number 13: 100%|██████████| 13/13 [00:05<00:00,  2.34batches/s]



Metrics: {'train_loss': 0.9279966017212531, 'test_loss': 1.8339825868606567, 'bleu': 9.2101, 'gen_len': 9.202}




  1%|          | 1/122 [00:40<1:21:50, 40.58s/it]

...

 99%|█████████▉| 121/122 [1:05:19<00:34, 34.14s/it]

For epoch 127: 
{Learning rate: [0.0017372889394730215]}


Test batch number 13: 100%|██████████| 13/13 [00:05<00:00,  2.28batches/s]



Metrics: {'train_loss': 0.009828381110429576, 'test_loss': 2.2549900458409238, 'bleu': 21.6738, 'gen_len': 9.4242}




100%|██████████| 122/122 [1:05:53<00:00, 32.40s/it]


### ---

In [11]:
trainer.train(epochs = config['epochs'] - trainer.current_epoch, auto_save=True, metric_for_best_model='bleu', metric_objective='maximize', log_step=1,
              saving_directory = config['new_model_dir'])

  0%|          | 0/127 [00:00<?, ?it/s]c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\torch\optim\lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


For epoch 128: 
{Learning rate: [0.0017132983766039783]}


Test batch number 13: 100%|██████████| 13/13 [00:04<00:00,  2.68batches/s]



Metrics: {'train_loss': 0.010737099127392426, 'test_loss': 2.2635114743159366, 'bleu': 21.0653, 'gen_len': 9.5909}




  1%|          | 1/127 [00:37<1:17:45, 37.03s/it]

...

 99%|█████████▉| 126/127 [1:11:59<00:33, 33.46s/it]

For epoch 254: 
{Learning rate: [3.3315773999033293e-07]}


Test batch number 13: 100%|██████████| 13/13 [00:04<00:00,  2.74batches/s]



Metrics: {'train_loss': 0.0059516656105500654, 'test_loss': 2.221930687244122, 'bleu': 21.1626, 'gen_len': 9.7121}




100%|██████████| 127/127 [1:12:28<00:00, 34.24s/it]


### Predictions and Evaluation

In [13]:
# initialize the transformation sequence
end_mark_fn = partial(add_end_mark)
fr_augmentation = TransformerSequences(remove_mark_space, delete_guillemet_space, end_mark_fn)

# let us get the test set
test_dataset = SentenceDataset(f"data/extractions/new_data/test_set.csv",
                                        tokenizer = tokenizer,
                                        cp1_transformer = fr_augmentation,
                                        cp2_transformer = fr_augmentation,
                                        truncation = True)

Let us make the evaluation and print the predicted sentences.

In [14]:
# evaluation with test set
df_ft_to_wf = trainer.evaluate(test_dataset)

Evaluation batch number 13: 100%|██████████| 13/13 [00:06<00:00,  2.16batches/s]


In [15]:
df_ft_to_wf[1].tail(10)

,original_sentences,translations,predictions
188,Donne le travail à un autre!,Joxal kenn liggéey bi!,Mayal ñépp benn!
189,Cet homme qui a été.,Góor gii demoon.,Gor gii dem.
190,Dites-lui.,Nileen ka.,Nileen leen.
191,Il est là.,Ma ŋgoogule foofu.,Mi ŋgi fi.
192,Tu vois cet homme là-bas?,Gis ŋga nit kale?,Gis ŋga xale bee?
193,L'homme est parti je crois!,Ma defe góor gi dem na!,"Góor gi dem na, ma defe!"
194,J'ai aperçu un baobab.,Séen naa ag guy.,Séen naa aw fas.
195,Celui-ci serait parti.,Kii dafa demkoon.,Kii dafa demkoon.
196,Toutes les portes étaient ouvertes.,Bunt yi yépp a tëjju woon.,Yeneen xar yi yépp daw.
197,"C'est Fatim, aujourd'hui.","Faatim la, tay.",Faatim la soo demee.


In [16]:
# let us display 100 samples
pd.options.display.max_rows = 100
df_ft_to_wf[1].sample(100)

,original_sentences,translations,predictions
18,Afin que parte qui?,Ndax kan dem?,Ndax kan dem?
168,Tu as vu les autres amis?,Gis ŋga sa yeneen xarit yi?,Gis ŋga samay xarit yan?
63,Quelles femmes se sont égarées?,Jigéen ñan ñoo réer?,Jigéen ñan ñoo réer?
175,C'est ce que tu as dit que la jeune femme préf...,Li ŋga wax la ndaw si taamu.,Li ŋga wax loolu.
71,Moi-même je n'ai pas été.,Man mii demuma.,Man demuma.
86,Ils partent.,Dem nañu.,Dem nañu.
5,"Les gens n'étaient pas, habituellement, bavards.",Nit ñi daawuñu coow.,Góor ñi dem nañu.
118,Bonne soirée.,Naka ngon si.,Jaam nga fanane.
12,Tout autre mouton que tu vois.,Meneen xar mépp moo gis.,"Wax ji yépp, bañ-ŋga-ñëw la."
150,Une photo datant d'il y a soixante et neuf ans...,Nataalu juróom benn fukk ak juróom ñeent day w...,Waaw nataal bii de ay bunt yu dóomu-taal moo c...
